In [1]:
from nltk.tokenize import TreebankWordTokenizer
import tensorflow as tf
import numpy as np
import re
import string

In [2]:
with open('iliad_pope.txt', 'r') as f:
    iliad_raw = f.readlines()

In [7]:
line_regex = re.compile('  [a-zA-Z]')
footnote_regex = re.compile('\(\d+\)')
iliad_text_list = [footnote_regex.sub('',line.strip()) for line in iliad_raw if line_regex.match(line)]
start_ind = iliad_text_list.index("Achilles' wrath, to Greece the direful spring")
iliad_text_list = iliad_text_list[start_ind:-1]
iliad_text = ' '.join(iliad_text_list).translate(str.maketrans('', '', string.punctuation))

In [8]:
iliad_tokens = TreebankWordTokenizer().tokenize(iliad_text)


['Achilles', 'wrath', 'to', 'Greece', 'the', 'direful', 'spring', 'Of', 'woes', 'unnumberd', 'heavenly', 'goddess', 'sing', 'That', 'wrath', 'which', 'hurld', 'to', 'Plutos', 'gloomy', 'reign', 'The', 'souls', 'of', 'mighty', 'chiefs', 'untimely', 'slain', 'Whose', 'limbs', 'unburied', 'on', 'the', 'naked', 'shore', 'Devouring', 'dogs', 'and', 'hungry', 'vultures', 'tore', 'Since', 'great', 'Achilles', 'and', 'Atrides', 'strove', 'Such', 'was', 'the', 'sovereign', 'doom', 'and', 'such', 'the', 'will', 'of', 'Jove', 'Declare', 'O', 'Muse', 'in', 'what', 'illfated', 'hour', 'Sprung', 'the', 'fierce', 'strife', 'from', 'what', 'offended', 'power', 'Latonas', 'son', 'a', 'dire', 'contagion', 'spread', 'And', 'heapd', 'the', 'camp', 'with', 'mountains', 'of', 'the', 'dead', 'The', 'king', 'of', 'men', 'his', 'reverent', 'priest', 'defied', 'And', 'for', 'the', 'kings', 'offence', 'the', 'people', 'died', 'For', 'Chryses', 'sought', 'with', 'costly', 'gifts', 'to', 'gain', 'His', 'captive', 